In [ ]:
# import packages
import logging
import time
import requests
import os
import json
import pandas as pd
import csv
import datetime
import dateutil.parser
import unicodedata
import time
import multiprocessing as mp
import math
import requests
from itertools import cycle
from lxml.html import fromstring
from itertools import cycle
import numpy as np
import tqdm
from tqdm import tqdm
import gc
import gzip
import json
import sys
tqdm.pandas()
from datetime import datetime as dtime

In [ ]:
# put your result path here
collect_dir = "./" 

In [ ]:
# put your token here, it can take multiple tokens
bearer_tokens = [
    'token1',
    'token2',
    'token3'
] 


In [ ]:
# create headers for API
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token),
              "Cache-Control":"no-cache"}
    return headers

In [ ]:
# error class
class TooManyRequests(Exception):
    def __init__(self, message):
        super().__init__(message)
class ChangeBearerToken(Exception):
    def __init__(self, message):
        super().__init__(message)
class GeneralException(Exception):
    def __init__(self, message):
        super().__init__(message)  
class ServiceNotAvailable(Exception):
    def __init__(self, message):
        super().__init__(message)

In [ ]:
def connect_to_endpoint(url, headers, params, next_token = None):
    global response
    params['next_token'] = next_token
    time.sleep(3)
    response = requests.request("GET", url, headers = headers, params = params)
    if response.status_code != 200:
        if "TooManyRequests" in response.text:
            raise TooManyRequests(response.text)
        elif "UsageCapExceeded" in response.text:
            raise ChangeBearerToken(response.text)
        elif "Service Unavailable" in response.text or response.status_code == 503:
            raise ServiceNotAvailable(response.text)
        else:
            raise GeneralException(response.text)
    return response.json()

In [ ]:
# results will be saved as gz files in a monthly folder
def append_to_gz(merged_dicts, fileName):

    fout = gzip.open(fileName, "a")

    try:
        json_str = json.dumps(merged_dicts) + "\n"            
        json_bytes = json_str.encode('utf-8')           
        fout.write(json_bytes)   

    except IndexError:
        print("Tweets Written Failed") 
        
    fout.close()

In [ ]:
# function to request tweets
def get_tweets(bearer_token, folder_name, fname, startdate, enddate):
    global json_response;
    global response 
    global merged_dicts;

    headers = create_headers(bearer_token)
    max_results = 500 # define the max results that one page can collect. Change to a smaller number will increase the speed for collecting a page data, but will result in needing more pages.
    url = "https://api.twitter.com/2/tweets/search/all"
    
    # change this query params based on your own needs; you can define keywords/location/time etc.
    # for more information including limits, please read the official twitter api documents.
    query_params = {"query": "(asian OR Japanese OR thai OR korean OR asians OR filipino OR vietnamese OR nepal OR nigga OR niggas OR black (boy OR boys OR girl OR girls OR man OR men OR people OR ppl OR woman OR women) OR blm OR nigger OR negro OR blacks OR slavery OR #blacklivesmatter OR mexican OR hispanic OR latino OR latina OR mexicans OR puerto rican OR dominican OR cuban OR latinos OR border (security OR wall) OR latinx OR immigration OR immigrants OR illegals OR immigrant OR sanctuary OR migrants OR migrant OR undocumented OR illegal (aliens OR immigrants) OR afghanistan OR jewish OR taliban OR muslim OR jews OR yemeni OR muslims OR islam OR afghan OR racist OR racism OR racists OR poc OR minorities OR people of color OR interracial OR biracial OR sioux OR cherokee OR native (american OR americans) OR refugee OR whites OR nazis OR white (boy OR boys OR girl OR girls OR guy OR man OR men OR people OR ppl OR woman OR women) OR kkk OR redneck OR proud boys OR Caucasian OR black girl OR chinese) place_country:US has:geo lang:en",
                    'max_results': max_results,
                    'start_time': startdate,
                    'end_time': enddate,
                    'expansions': 'geo.place_id,author_id,in_reply_to_user_id,referenced_tweets.id,referenced_tweets.id.author_id,entities.mentions.username',
                    'tweet.fields': 'id,text,author_id,created_at,geo,lang,public_metrics,source,conversation_id,in_reply_to_user_id,referenced_tweets',
                    'user.fields': 'id,name,username,created_at,description,location,pinned_tweet_id,public_metrics,verified',
                    'place.fields': 'full_name,id,geo,name,place_type',
                    'next_token':{}}

    total_tweets = 0
    
    flag = True
    next_token = None
    page = 0;
    print("Start time: " + str(startdate) + " Current time: " + str(dtime.now()))

    while flag:
        page =  page + 1;
        json_response = connect_to_endpoint(url, headers, query_params, next_token)
        
        print("\n Page #: " + str(page), end=". ")
        print("Tweets Received: " + str(len(json_response['data'])), end=". ")
        
        append_to_gz(json_response, collect_dir + folder_name + '/'  + fname +".gz")
        
        while True:
            if 'next_token' in json_response['meta']:
                next_token = json_response['meta']['next_token']
            else:
                flag = False
                next_token = None
            break
        
        total_tweets = total_tweets + len(json_response['data'])
         
    print("Total number of results: ", str(total_tweets))


In [ ]:
# note that for the listed error exceptions here, the current page will be restarted, which means duplicate tweets may be collected. Remove duplicates during the post process.
def do_everything(folder_name, fname, start, end, iii):

    iii = iii
    while True:
            try:
                if not os.path.isdir(collect_dir + folder_name):
                    os.makedirs(collect_dir + folder_name)
                get_tweets(bearer_tokens[iii], folder_name, fname, start, end)
            except ChangeBearerToken as e: 
                print('Change bearer token: ' + str(e) + ' restart this page')
                iii = iii + 1
                if iii == len(bearer_tokens):
                    iii = 0
                    print("All tokens are used for current time period")
                continue
            except GeneralException as e:
                print('General exception: ' + str(e) + ' restart this page')
                continue
            except TooManyRequests as e:
                print('Too many requests: ' + str(e) + ' restart this page')
                time.sleep(10)
                continue
            except ServiceNotAvailable as e:
                print('Service not available: ' + str(e) + ' restart this page')
                continue
            break
    return iii   
 

In [ ]:
# put the timeframe you want to collect tweets from.
date = pd.date_range(start='2022-01-01T00:00:00.000Z', periods=745, freq='H') 
date = [datetime.datetime.strftime(i, "%Y-%m-%dT%H:00:00Z") for i in date]

In [ ]:
# main code
iii = 0

for i in range(len(date) - 1):
        connected = False
        while not connected:
            try:
                start_time = date[i];
                end_time = date[i + 1];

                iii = do_everything( start_time[:7], start_time[8:10], start_time, end_time, iii)
                print("current bearer token is " + str(iii))
                connected = True
            except Exception as e:
                print('figure out why' + str(e) + "*********************")
                
                # date-hour that with issues will be saved to a file.                
                with open ("C:/Users/xyue/collected_data_full_academic/log_witherros.csv",'a', newline='') as filedata:
                    writer = csv.writer(filedata)
                    row = [str(start_time)]
                    writer.writerow(row)
                filedata.close()
                
                connected = True